In [56]:
import pandas as pd
import numpy as np
import math

from tqdm.notebook import tqdm

In [2]:
path = 'C:/Users/Rusla/Documents/GitHub/Пушкинская карта - стажировка/Sample/'

In [3]:
users = pd.read_csv(path + 'users_full.csv', sep=';', error_bad_lines=False)

In [4]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2879681 entries, 0 to 2879680
Data columns (total 9 columns):
 #   Column              Dtype  
---  ------              -----  
 0   create_date         int64  
 1   user_id             object 
 2   user_birth          object 
 3   create_date.1       object 
 4   user_phone_details  object 
 5   buyer_mobile_phone  object 
 6   region              object 
 7   age                 int64  
 8   user_region         float64
dtypes: float64(1), int64(2), object(6)
memory usage: 197.7+ MB


In [5]:
users.sample(3)

,create_date,user_id,user_birth,create_date.1,user_phone_details,buyer_mobile_phone,region,age,user_region
2287623,1630635009,3f25e617bfd6118a62d99345b9f3ab1a,2004-11-20,2021-11-07,"Android, 11 (SDK 30)",5a5ddfd5ac45252c6009ebc05279c847,Иркутская область,17,38.0
1127258,1636471269,72c276ee54016b7e8477b4189520671e,2004-09-18,2021-11-09,"Android, 11 (SDK 30)",34dfb494581bc85d6fedfcff6dcefcc9,NaN,17,NaN
2254336,1636897445,29952dec11c5e5a4ec3224ccefbecc7d,2004-11-20,2021-11-03,"Android, 9 (SDK 28)",efe21e841cb463b182e4698abbbf0cb0,Кемеровская область – Кузбасс,17,42.0


In [6]:
users = users.astype({"age": "object", "user_region": "object"})

In [7]:
import json 
expl = [json.loads(line) for line in open(path + 'tickets2.json', 'r', encoding='utf-8', errors = 'ignore')]

In [8]:
tickets = pd.DataFrame(expl)

In [9]:
tickets.sample(3)

,payment_date,comment,session_event_id,session_date,session_params,owner,refund_date,buyer_mobile_phone,refund_reason,payment_amount,visitor_first_name,refund_ticket_price,create_date,status,payment_ticket_price,session_organization_id,visit_date
229107,1636478522,Погашение билета ООО ДТЗК bileter.ru,1319545,1637942400,Партер р.11 м.46,7840404467,NaN,b6afe4eddee70c8b617968532c7a1bc4,None,2100,,None,1636478762107468634,visited,2100,21922,1.638172e+09
122661,1636635762,,1243074,1638108000,"«Большая сцена», Партер, 6 ряд, 9 место",7703747085,NaN,053ced106f641c95bd2f4e3ccfb35921,None,900.0,,None,1636635762139527514,active,900.0,7366,NaN
631342,1637093836,,345853,1641744000,"Партер, Ряд: 6, Место: 194",7802816445,NaN,301dcec2628e2dcb3288b3f8c92a9b9e,None,2200.00,,None,1637093836808034187,active,2200.00,7603,NaN


In [10]:
tickets[tickets['buyer_mobile_phone'].isna()]

,payment_date,comment,session_event_id,session_date,session_params,owner,refund_date,buyer_mobile_phone,refund_reason,payment_amount,visitor_first_name,refund_ticket_price,create_date,status,payment_ticket_price,session_organization_id,visit_date


In [11]:
tickets.visitor_first_name.nunique() # можно было попробовать пол по именм, но имён у нас всего 11 тысяч из 1,2 млн.

11791

# Добавим данные пользователя в дата сет с купленными билетами

In [12]:
tickets = pd.merge(tickets,users,on=['buyer_mobile_phone'], how='left')

In [13]:
tickets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 854330 entries, 0 to 854329
Data columns (total 25 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   payment_date             854330 non-null  int64  
 1   comment                  854330 non-null  object 
 2   session_event_id         854330 non-null  object 
 3   session_date             854330 non-null  int64  
 4   session_params           854330 non-null  object 
 5   owner                    854330 non-null  object 
 6   refund_date              25117 non-null   float64
 7   buyer_mobile_phone       854330 non-null  object 
 8   refund_reason            25117 non-null   object 
 9   payment_amount           854330 non-null  object 
 10  visitor_first_name       854330 non-null  object 
 11  refund_ticket_price      24409 non-null   object 
 12  create_date_x            854330 non-null  int64  
 13  status                   854330 non-null  object 
 14  paym

In [18]:
tickets_2 = tickets[tickets['user_id'].notna() == True]

In [19]:
tickets_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 530576 entries, 1 to 854329
Data columns (total 25 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   payment_date             530576 non-null  int64  
 1   comment                  530576 non-null  object 
 2   session_event_id         530576 non-null  object 
 3   session_date             530576 non-null  int64  
 4   session_params           530576 non-null  object 
 5   owner                    530576 non-null  object 
 6   refund_date              16455 non-null   float64
 7   buyer_mobile_phone       530576 non-null  object 
 8   refund_reason            16455 non-null   object 
 9   payment_amount           530576 non-null  object 
 10  visitor_first_name       530576 non-null  object 
 11  refund_ticket_price      16025 non-null   object 
 12  create_date_x            530576 non-null  int64  
 13  status                   530576 non-null  object 
 14  paym

In [36]:
ticket_users = tickets_2.user_id.unique()

In [38]:
ticket_users[1]

'68f0a5ca2209fedf6d574d74940e55a5'

In [22]:
cliks_add = pd.read_csv(path + 'cliks_add.csv', sep=';', error_bad_lines=False)

In [23]:
cliks_add.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5747389 entries, 0 to 5747388
Data columns (total 17 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   create_time              object 
 1   create_date              object 
 2   user_phone_details       object 
 3   buyer_mobile_phone       object 
 4   user_id                  object 
 5   url                      object 
 6   session_id               float64
 7   session_name             object 
 8   organization_id          float64
 9   organization_name        object 
 10  session_identity         object 
 11  Org_region_number        float64
 12  org_category             object 
 13  age                      float64
 14  user_region              float64
 15  user_phone_details_id    object 
 16  user_phone_details_id_2  int64  
dtypes: float64(5), int64(1), object(11)
memory usage: 745.4+ MB


In [42]:
cliks_and_tickets = cliks_add.copy()

In [43]:
cliks_and_tickets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5747389 entries, 0 to 5747388
Data columns (total 17 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   create_time              object 
 1   create_date              object 
 2   user_phone_details       object 
 3   buyer_mobile_phone       object 
 4   user_id                  object 
 5   url                      object 
 6   session_id               float64
 7   session_name             object 
 8   organization_id          float64
 9   organization_name        object 
 10  session_identity         object 
 11  Org_region_number        float64
 12  org_category             object 
 13  age                      float64
 14  user_region              float64
 15  user_phone_details_id    object 
 16  user_phone_details_id_2  int64  
dtypes: float64(5), int64(1), object(11)
memory usage: 745.4+ MB


In [53]:
cliks_and_tickets = pd.DataFrame()

In [48]:
cliks_add_users = cliks_add.user_id.unique()

In [50]:
cliks_add_users[1]

'b2560eaa8bc23dbba5f24aa5075b89ed'

In [68]:
%%time
users_t_and_c = []
for i in tqdm(cliks_add_users):
    if i in ticket_users:
        users_t_and_c.append(i)

  0%|          | 0/272442 [00:00<?, ?it/s]

Wall time: 3h 34min 56s


In [72]:
len(users_t_and_c)

137156

In [ ]:
%%time
for i in tqdm(users_t_and_c):
    cliks_and_tickets = pd.concat([cliks_and_tickets, cliks_add[cliks_add['user_id'] == i]])

  0%|          | 0/137156 [00:00<?, ?it/s]

In [70]:
cliks_and_tickets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39522 entries, 0 to 3863061
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   create_time              39522 non-null  object 
 1   create_date              39522 non-null  object 
 2   user_phone_details       39522 non-null  object 
 3   buyer_mobile_phone       39522 non-null  object 
 4   user_id                  39522 non-null  object 
 5   url                      39522 non-null  object 
 6   session_id               39406 non-null  float64
 7   session_name             39406 non-null  object 
 8   organization_id          39406 non-null  float64
 9   organization_name        39406 non-null  object 
 10  session_identity         39406 non-null  object 
 11  Org_region_number        39098 non-null  float64
 12  org_category             36980 non-null  object 
 13  age                      39522 non-null  float64
 14  user_region         

In [61]:
cliks_and_tickets.sample(3)

ValueError: a must be greater than 0 unless no samples are taken

In [62]:
df = pd.concat([cliks_and_tickets, cliks_add[cliks_add['user_id'] == 'b2560eaa8bc23dbba5f24aa5075b89ed']])

In [63]:
df

,create_time,create_date,user_phone_details,buyer_mobile_phone,user_id,url,session_id,session_name,organization_id,organization_name,session_identity,Org_region_number,org_category,age,user_region,user_phone_details_id,user_phone_details_id_2
4,2021-11-28 21:25:13,2021-11-28,"iOS, iOS 15.0.2",a1f60fad21d72c6dcb5fd7fe84fb3ad7,b2560eaa8bc23dbba5f24aa5075b89ed,https://tickets.hermitagemuseum.org/event/7E06...,1309020.0,Экспозиции главного музейного комплекса Эрмитажа,2217.0,ФГБУК «Государственный Эрмитаж»,Экспозиции,78.0,Музеи и галереи,22.0,77.0,"iOS, 15",215
5,2021-11-28 21:25:13,2021-11-28,"iOS, iOS 15.0.2",a1f60fad21d72c6dcb5fd7fe84fb3ad7,b2560eaa8bc23dbba5f24aa5075b89ed,https://tickets.hermitagemuseum.org/event/7E06...,1309020.0,Экспозиции главного музейного комплекса Эрмитажа,2217.0,ФГБУК «Государственный Эрмитаж»,Экспозиции,78.0,Музеи и галереи,22.0,77.0,"iOS, 15",215
565977,2021-11-30 00:25:55,2021-11-30,"iOS, iOS 15.0.2",a1f60fad21d72c6dcb5fd7fe84fb3ad7,b2560eaa8bc23dbba5f24aa5075b89ed,https://spa.profticket.ru/customer/80/shows/92...,1359726.0,Концерт «Jazz-Classic-XXI»,6418.0,ГБУК «Санкт-Петербургская филармония джазовой ...,Концерт,78.0,Концертные площадки,22.0,77.0,"iOS, 15",215
565978,2021-11-30 00:25:55,2021-11-30,"iOS, iOS 15.0.2",a1f60fad21d72c6dcb5fd7fe84fb3ad7,b2560eaa8bc23dbba5f24aa5075b89ed,https://spa.profticket.ru/customer/80/shows/92...,1359726.0,Концерт «Jazz-Classic-XXI»,6418.0,ГБУК «Санкт-Петербургская филармония джазовой ...,Концерт,78.0,Концертные площадки,22.0,77.0,"iOS, 15",215
565979,2021-11-30 00:31:25,2021-11-30,"iOS, iOS 15.0.2",a1f60fad21d72c6dcb5fd7fe84fb3ad7,b2560eaa8bc23dbba5f24aa5075b89ed,https://ticket.rusmuseum.ru/event/9CCAC74276F6...,1296265.0,Программа «Музыкальные четверги в Мраморном дв...,4524.0,ФГБУК «Государственный Русский музей»,Программа,78.0,Музеи и галереи,22.0,77.0,"iOS, 15",215
565980,2021-11-30 00:31:25,2021-11-30,"iOS, iOS 15.0.2",a1f60fad21d72c6dcb5fd7fe84fb3ad7,b2560eaa8bc23dbba5f24aa5075b89ed,https://ticket.rusmuseum.ru/event/9CCAC74276F6...,1296265.0,Программа «Музыкальные четверги в Мраморном дв...,4524.0,ФГБУК «Государственный Русский музей»,Программа,78.0,Музеи и галереи,22.0,77.0,"iOS, 15",215
565981,2021-11-30 00:35:13,2021-11-30,"iOS, iOS 15.0.2",a1f60fad21d72c6dcb5fd7fe84fb3ad7,b2560eaa8bc23dbba5f24aa5075b89ed,https://ticket.rusmuseum.ru/event/82DC859FAE69...,1333915.0,Экскурсия «История и современность. На примере...,4524.0,ФГБУК «Государственный Русский музей»,Экскурсия,78.0,Музеи и галереи,22.0,77.0,"iOS, 15",215
565982,2021-11-30 00:35:13,2021-11-30,"iOS, iOS 15.0.2",a1f60fad21d72c6dcb5fd7fe84fb3ad7,b2560eaa8bc23dbba5f24aa5075b89ed,https://ticket.rusmuseum.ru/event/82DC859FAE69...,1333915.0,Экскурсия «История и современность. На примере...,4524.0,ФГБУК «Государственный Русский музей»,Экскурсия,78.0,Музеи и галереи,22.0,77.0,"iOS, 15",215
1841369,2021-11-04 15:29:27,2021-11-04,"iOS, iOS 15.0.2",a1f60fad21d72c6dcb5fd7fe84fb3ad7,b2560eaa8bc23dbba5f24aa5075b89ed,https://tickets.hermitagemuseum.org/event/7E06...,1309020.0,Экспозиции главного музейного комплекса Эрмитажа,2217.0,ФГБУК «Государственный Эрмитаж»,Экспозиции,78.0,Музеи и галереи,22.0,77.0,"iOS, 15",215
1841370,2021-11-04 15:29:27,2021-11-04,"iOS, iOS 15.0.2",a1f60fad21d72c6dcb5fd7fe84fb3ad7,b2560eaa8bc23dbba5f24aa5075b89ed,https://tickets.hermitagemuseum.org/event/7E06...,1309020.0,Экспозиции главного музейного комплекса Эрмитажа,2217.0,ФГБУК «Государственный Эрмитаж»,Экспозиции,78.0,Музеи и галереи,22.0,77.0,"iOS, 15",215
